# Type Ia Supernova Hubble Diagram

## Scientific Background
Since the early 20th century, astronomers have known the universe was expanding by observing the redshift of stellar objects. It wasn't until 1998 that two teams of astronomers were able to deduce that not only was the universe expanding, but this rate of expansion was changing. This discovery wasn't a new idea to the scientific world, but theorists were shocked at the results of these studies.

### Proposed Cosmological Models
Since we knew the universe was expanding, there were three options for the universe's behavior:
1. __It was slowing down__ due to the gravitational force from all the matter in the universe, eventually collapsing the universe back in on itself
2. __It wasn't accelerating at all__ and the universe would continue to expand at a constant rate
3. __It was speeding up,__ but this wouldn't make much sense with what we knew at the time. What could possibly be causing that?

What we found in 1998 was that, shockingly enough, option 3 was correct and the universe's expansion was actually speeding up. The cause of this acceleration was dubbed "dark energy" and is still being heavily investigated today. The results of these studies would go on to win the 2011 Nobel Prize in Physics.

## The Methods of This Study
The core idea of this study was that if you know how both bright a star is supposed to be and what wavelengths of light it should be giving off, then you can compare these expected values with your own measurements to find how far that light traveled and how fast the star was moving away from you. The problem here is that very few stellar objects are "standard candles" like this, which makes data sparse and hard to gather. Luckily, one particular kind of supernova — the type Ia (or one-a) supernova — has this exact quality, reaching nearly the same peak brightness regardless of the particular star, as long as it happens to a white dwarf under cerain circumstances. With this we have our two main data points.

### Distance Modulus
The distance modulus is a conveniant method of storing distance data in astronomy. It represents the difference between expected and measured magnitude of brightness. In the same way that a flashlight appears dimmer as it moves farther away, a type Ia supernova works the same way since we know how bright it actually is (about -19.3 mag). When gathering data on observed brightness, the distance modulus is the simplest way to convey the distance from type Ia SNe, but the distance the light traveled to get to us (called luminosity distance) is given by this formula.

$$
\mu = 5 \log_{10}\left(\frac{D_L}{10\,\text{pc}}\right)
$$

Where $\mu$ is the distance modulus and $D_L$ is the luminosity distance in parsecs. Rearranging this equation to solve for $D_L$ in megaparsecs (a more common distance metric) gives you this.

$$
D_L\,\text{[Mpc]} = 10^{(\mu - 25)/5}.
$$

Going back and forth between modulus and luminosity distance is key for this kind of data analysis. It is also worth noting that luminosity distance is not the actual measure of how far away an object is (we can never know that) but how far the light we observe has traveled to get to us. It tells how far away on object once was.

### Redshift
The redshift seen from a star is caused by it moving either toward or away from an observer. Because of the doppler effect, the light it emits gets pushed to a shorter or longer wavelength depending on the direction of its relative motion. In our case with an expanding universe, stellar objects practically always have redshifted rather than blueshifted light. the redshift is expressed as a factor of the measured light frequency divided by the expected and this factor can easily be converted to a relative velocity between earth and what's being measured. However, what we're doing here does not actually need that velocity, so this conversion is not necessary for us. 

The redshift of type Ia SNe adn be found with a few different methods, including finding the redshift of the galaxy it is in or doing spectroscopic measurements of the supernova itself. The data set we pull from is a compilation of several different data sets on type Ia SNe gathered by the Supernova Cosmology Project, so the patricular methods might vary across this data set.


## What I'll be Doing
Using publically available data on distance modulus and redshift for 580 type Ia SNe, I plan to find not only the current rate of expansion in the universe, but also the ratio of matter density to dark energy density in the universe that corresponds to the accleration of this expansion.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

import SNe_Hubble as sne

In [ ]:
data = sne.load_modulus_data()
subset = data[: sne.PLOT_ENTRY_LIMIT]

if len(subset) == 0:
    raise RuntimeError("No supernova data available to process.")

modulus = subset["Distance_Modulus"]
modulus_error = subset["Distance_Modulus_Error"]
redshift = subset["Redshift"]

(distances, distance_errors) = sne.modulus_to_distance(
    modulus,
    modulus_error,
)

In [ ]:
print(f"Loaded {len(data)} supernovae using SciPy {scipy.__version__}.")
print(f"Plotting first {len(subset)} entries to {sne.PLOT_OUTPUT_PATH.name}.")
print("First entry as a quick check:")
print(
    f"{subset['Supernova_Name'][0]} | "
    f"mu = {modulus[0]:.3f} +/- {modulus_error[0]:.3f} -> "
    f"d = {distances[0]:.3e} +/- {distance_errors[0]:.3e} pc"
)

In [ ]:
sne.plot_modulus_vs_redshift(
    redshift,
    modulus,
    modulus_error,
    sne.PLOT_OUTPUT_PATH,
)
print(f"Saved plot to {sne.PLOT_OUTPUT_PATH.resolve()}")

Step Two

In [ ]:
best_fit_h0 = None
best_fit_h0_err = None
best_fit_h0_km_s_mpc = None
best_fit_h0_km_s_mpc_err = None

low_z_mask = redshift < sne.LOW_Z_THRESHOLD

filtered_low_z = None
filtered_low_d = None
filtered_low_d_err = None
low_z_rejected_points = None

if np.any(low_z_mask):
    low_z = redshift[low_z_mask]
    low_d = distances[low_z_mask]
    low_d_err = distance_errors[low_z_mask]

    low_valid_mask = (
        np.isfinite(low_z)
        & np.isfinite(low_d)
        & np.isfinite(low_d_err)
        & (low_d_err >= 0.0)
    )

    def _low_z_model(z_subset, d_subset, err_subset):
        h0, _ = sne.fit_hubble_constant(z_subset, d_subset, err_subset)
        return sne.hubble_law(z_subset, h0)

    (inlier_mask, outlier_mask) = sne._sigma_clip_outliers(
        low_z,
        low_d,
        low_d_err,
        low_valid_mask,
        _low_z_model,
        min_required_points=2,
        sigma_threshold=sne.LOW_Z_SIGMA_CLIP_THRESHOLD,
        max_iterations=sne.LOW_Z_SIGMA_CLIP_MAX_ITER,
    )

    if not np.any(low_valid_mask):
        print(
            "No finite low-z entries survived quality checks; "
            "skipping low-z fit and plot."
        )
    else:
        filtered_low_z = low_z[inlier_mask]
        filtered_low_d = low_d[inlier_mask]
        filtered_low_d_err = low_d_err[inlier_mask]
        num_outliers = int(np.count_nonzero(outlier_mask))
        low_z_rejected_points = None

        if filtered_low_z.size >= 2:
            if num_outliers:
                low_z_rejected_points = (
                    low_z[outlier_mask],
                    low_d[outlier_mask],
                    low_d_err[outlier_mask],
                )
                print(
                    f"Removed {num_outliers} low-z outlier(s) "
                    f"using {sne.LOW_Z_SIGMA_CLIP_THRESHOLD:.0f} sigma residual clipping."
                )
        else:
            filtered_low_z = low_z[low_valid_mask]
            filtered_low_d = low_d[low_valid_mask]
            filtered_low_d_err = low_d_err[low_valid_mask]
            low_z_rejected_points = None
            if num_outliers:
                print(
                    "Not enough low-z points after clipping; "
                    "using all valid measurements for the H0 fit instead."
                )
else:
    print(
        f"No entries below redshift {sne.LOW_Z_THRESHOLD}; "
        "skipping low-z plot."
    )

In [ ]:
if filtered_low_z is not None and filtered_low_z.size > 0:
    try:
        (best_fit_h0, best_fit_h0_err) = sne.fit_hubble_constant(
            filtered_low_z,
            filtered_low_d,
            filtered_low_d_err,
        )
        (
            best_fit_h0_km_s_mpc,
            best_fit_h0_km_s_mpc_err,
        ) = sne.to_km_s_per_mpc(
            best_fit_h0,
            best_fit_h0_err,
        )
        print(
            "Best-fit H0 for low-z subset: "
            f"{best_fit_h0:.3e} pc^-1 "
            f"(+/- {best_fit_h0_err:.3e}) -> "
            f"{best_fit_h0_km_s_mpc:.2f} km s^-1 Mpc^-1"
            + (
                f" +/- {best_fit_h0_km_s_mpc_err:.2f}"
                if best_fit_h0_km_s_mpc_err is not None
                else ""
            )
        )
    except Exception as exc:
        best_fit_h0 = None
        best_fit_h0_err = None
        best_fit_h0_km_s_mpc = None
        best_fit_h0_km_s_mpc_err = None
        print(f"Could not fit H0 for low-z subset: {exc}")

In [ ]:
if filtered_low_z is not None and filtered_low_z.size > 0:
    if best_fit_h0 is not None:
        sne.plot_distance_vs_redshift(
            filtered_low_z,
            filtered_low_d,
            filtered_low_d_err,
            sne.LOW_Z_PLOT_OUTPUT_PATH,
            best_fit_h0=best_fit_h0,
            best_fit_h0_error=best_fit_h0_err,
            best_fit_h0_km_s_mpc=best_fit_h0_km_s_mpc,
            best_fit_h0_km_s_mpc_error=best_fit_h0_km_s_mpc_err,
            rejected_points=low_z_rejected_points,
        )
    else:
        sne.plot_distance_vs_redshift(
            filtered_low_z,
            filtered_low_d,
            filtered_low_d_err,
            sne.LOW_Z_PLOT_OUTPUT_PATH,
        )

    print(
        f"Saved low-z (< {sne.LOW_Z_THRESHOLD}) distance plot to "
        f"{sne.LOW_Z_PLOT_OUTPUT_PATH.resolve()}"
    )

In [ ]:
h0_for_cosmo = best_fit_h0 if best_fit_h0 is not None else sne.DEFAULT_H0_PC_INV

if best_fit_h0 is None:
    print(
        "No reliable low-z H0 fit; defaulting to "
        f"{sne.DEFAULT_H0_KM_S_MPC:.1f} km s^-1 Mpc^-1 "
        "(converted to pc^-1) for cosmological optimization."
    )

In [ ]:
cosmo_valid_mask = (
    np.isfinite(redshift)
    & np.isfinite(distances)
    & np.isfinite(distance_errors)
    & (redshift >= 0.0)
    & (distances > 0.0)
    & (distance_errors >= 0.0)
)

filtered_redshift = None
filtered_distances = None
filtered_distance_errors = None
filtered_modulus = None
filtered_modulus_error = None
cosmo_rejected_points = None

if not np.any(cosmo_valid_mask):
    raise RuntimeError("No valid entries available for cosmological fit.")


def _cosmo_model(z_subset, d_subset, err_subset):
    (omega_m, omega_lambda) = sne.fit_density_parameters(
        z_subset,
        d_subset,
        err_subset,
        h0_for_cosmo,
    )
    return sne.predict_luminosity_distance(
        z_subset,
        omega_m,
        omega_lambda,
        h0_for_cosmo,
    )


try:
    (cosmo_inlier_mask, cosmo_outlier_mask) = sne._sigma_clip_outliers(
        redshift,
        distances,
        distance_errors,
        cosmo_valid_mask,
        _cosmo_model,
        min_required_points=3,
        sigma_threshold=sne.COSMO_SIGMA_CLIP_THRESHOLD,
        max_iterations=sne.COSMO_SIGMA_CLIP_MAX_ITER,
    )
except Exception as exc:
    print(f"Could not run cosmological sigma clipping: {exc}")
    cosmo_inlier_mask = cosmo_valid_mask.copy()
    cosmo_outlier_mask = np.zeros_like(redshift, dtype=bool)

cosmo_filtered_mask = cosmo_inlier_mask.copy()
num_cosmo_inliers = int(np.count_nonzero(cosmo_filtered_mask))
num_cosmo_outliers = int(np.count_nonzero(cosmo_outlier_mask))
use_all_cosmo_data = False

if num_cosmo_inliers < 3:
    cosmo_filtered_mask = cosmo_valid_mask
    use_all_cosmo_data = True
    if num_cosmo_outliers:
        print(
            "Not enough cosmological points after clipping; "
            "using all valid entries for the Ω fit instead."
        )
    num_cosmo_outliers = 0

if num_cosmo_outliers and not use_all_cosmo_data:
    cosmo_rejected_points = (
        redshift[cosmo_outlier_mask],
        modulus[cosmo_outlier_mask],
        modulus_error[cosmo_outlier_mask],
    )
    print(
        f"Removed {num_cosmo_outliers} cosmological outlier(s) "
        f"using {sne.COSMO_SIGMA_CLIP_THRESHOLD:.0f} sigma residual clipping."
    )

filtered_redshift = redshift[cosmo_filtered_mask]
filtered_distances = distances[cosmo_filtered_mask]
filtered_distance_errors = distance_errors[cosmo_filtered_mask]
filtered_modulus = modulus[cosmo_filtered_mask]
filtered_modulus_error = modulus_error[cosmo_filtered_mask]

In [ ]:
try:
    (omega_m_fit, omega_lambda_fit) = sne.fit_density_parameters(
        filtered_redshift,
        filtered_distances,
        filtered_distance_errors,
        h0_for_cosmo,
    )
    print(
        "Best-fit density parameters from cosmological d_L(z): "
        f"Omega_M = {omega_m_fit:.3f}, Omega_Lambda = {omega_lambda_fit:.3f}"
    )
    sne.plot_cosmological_modulus_fit(
        filtered_redshift,
        filtered_modulus,
        filtered_modulus_error,
        omega_m_fit,
        omega_lambda_fit,
        h0_for_cosmo,
        sne.COSMO_MODULUS_PLOT_OUTPUT_PATH,
        rejected_points=cosmo_rejected_points,
    )
    print(
        "Saved cosmological distance-modulus plot to "
        f"{sne.COSMO_MODULUS_PLOT_OUTPUT_PATH.resolve()}"
    )
except Exception as exc:
    print(f"Could not perform cosmological fit: {exc}")